In [2]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load dataset and split them into train and test set

In [3]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/'
#####################   Modify above path   #########################
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)
X_dis=X
y_dis=y

### Use GridsearchCV to determine Parameters

In [6]:
# from sklearn.model_selection import GridSearchCV
# svm_parm_gs = {'C':[0.01,0.1,1,50],
#               'kernel':['rbf', 'linear'],
#               'degree':[2,4],
#               'decision_function_shape': ['ovr', 'ovo'],
#               'gamma': [0.1,0.01,0.005],
#                 }

# svm_model_gs = SVC()

# svm_gsearch = GridSearchCV(estimator =svm_model_gs,
#                    param_grid=svm_parm_gs,
#                    scoring ='accuracy',
#                    cv = 3,
#                    n_jobs = 2,
#                    verbose=3,
#                    return_train_score=True
#                    )
# svm_gsearch.fit(train_x_dis,train_y_dis)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 14.4min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 63.1min
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed: 145.7min
[Parallel(n_jobs=2)]: Done 288 out of 288 | elapsed: 147.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=2,
             param_grid={'C': [0.01, 0.1, 1, 50],
                         'decision_function_shape': ['ovr', 'ovo'],
                         'degree': [2, 4], 'gamma': [0.1, 0.01, 0.005],
                         'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='accuracy', verbose=3)

In [7]:
# from sklearn.metrics import accuracy_score  
# print("Training dataset Accuracy")
# train_preds = svm_gsearch.predict(train_x_dis)
# train_accuracy = accuracy_score(train_y_dis, train_preds) 
# print(train_accuracy)

Training dataset Accuracy
1.0


In [8]:
# print("Testing dataset Accuracy")
# test_preds = svm_gsearch.predict(test_x_dis)
# test_accuracy = accuracy_score(test_y_dis, test_preds) 
# print(test_accuracy)

Testing dataset Accuracy
0.474


In [9]:
# svm_gsearch.best_params_

{'C': 0.1,
 'decision_function_shape': 'ovr',
 'degree': 2,
 'gamma': 0.1,
 'kernel': 'linear'}

### Use K-Fold to estimate the accuracy (50.04%)

In [4]:
svm_model_final =SVC(C=0.1,decision_function_shape='ovr',degree=2,gamma=0.1,kernel='linear')

In [13]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(svm_model_final, X_dis,y_dis, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 50.04% (2.85%)


### Fit the final model and save it

In [5]:
t0 = time.time()
svm_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

done in 28.614s


In [15]:
# joblib.dump(svm_model_final,'svm_model_final.m')#please change the path

['svm_model_final.m']

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/svm_model_final.m')